# Solubility of uranium for changing pH

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Mar 31th, 2022</p>

This tutorial shows an example of the calculation of uranium solubility at changing pH values in groundwater. This example uses the thermodynamic database `psinagra-12-07`, which is based on the [Nagra/PSI Chemical Thermodynamic Data Base 01/01] (https://www.psi.ch/de/les/database). It supports the ongoing safety assessments for the planned low and intermediate-level (L/ILW) and high-level (HLW) radioactive waste repositories in Switzerland.

```{note}

This tutorial is joint work with Dan Miron, PSI.

If your main interest is in calculating thermodynamic properties rather than modeling chemical equilibria and kinetics, you should check out [ThermoFun] (https://thermohub.org/thermofun/thermofun/), an excellent project dedicated to this task.
```

First, we set up the chemical system with the aqueous phase only.

In [1]:
from reaktoro import *
import numpy as np
import math

# Define the Thermofun database
db = ThermoFunDatabase ("psinagra-12-07")

# Define the aqueous phase
solution = AqueousPhase(speciate("Al C Ca Cl Fe H K Mg N Na O P S Si U"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

# Define chemical system by providing database and aqueous phase
system = ChemicalSystem(db, solution)

The equilibrium specifications given below indicate that the equilibrium calculations are performed for fixed T, P and pH. The corresponding equilibrium conditions and the equilibrium solver are initialized with the instance `specs`.

In [2]:
# Specify conditions to be satisfied at chemical equilibrium
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()

# Define conditions to be satisfied at the chemical equilibrium state
conditions = EquilibriumConditions(specs)
conditions.temperature(25.0, "celsius")
conditions.pressure(1.0, "bar")

# Define the equilibrium solver
solver = EquilibriumSolver(specs)

The initial chemical state is defined according to the following recipe:

* 1000 g of water,
* 1e-5 mol of H<sub>3</sub>PO<sub>4</sub>@,
* 1e-5 mol of UO<sub>2</sub>(SO<sub>4</sub>)@, and
* 0.1 g of CO<sub>2</sub>@.

In [3]:
# Define initial equilibrium state
state = ChemicalState(system)
state.set("H2O@"     , 1e3,  "g")
state.set("CO2@"     , 1e-1, "g")
state.set("H3PO4@"   , 1e-5, "mol")
state.set("UO2(SO4)@", 1e-5, "mol")

The amount of uranium can be calculated based on the chemical properties of this chemical state. We define the range of pH values and the list of uranium-containing species of interest for our evaluation.

In [4]:
# Calculate the amount of uranium element
prop = ChemicalProps(state)
bU = prop.elementAmount("U")[0]

# Defined an auxiliary array of pH values
pHs = np.linspace(5, 10, num=41)

# Create list of species names, list of Species objects, and auxiliary amounts array
species_list_str = "UO2+2 UO2OH+ UO2(OH)2@ UO2CO3@ (UO2)3CO3(OH)3+ (UO2)2(OH)+3 " \
                   "UO2(CO3)3-4 UO2(CO3)2-2 (UO2)2(OH)2+2 (UO2)3(OH)5+ (UO2)4(OH)7+"
species_list = SpeciesList(species_list_str)
amounts = np.zeros(species_list.size())

# Define dataframe to collect amount of the selected species
import pandas as pd
columns = ["pH"] + ["amount_" + name for name in species_list_str.split()]
df = pd.DataFrame(columns=columns)

Finally, we run simulations in the for loop on the pH values provided for the instance of the equilibrium conditions.

In [5]:
for pH in pHs:

    # Set the value of pH for the current equilibrium calculations
    conditions.pH(pH)

    # Equilibrate the initial state with given conditions and component amounts
    res = solver.solve(state, conditions)

    # Otherwise, calculate U(VI) Speciation, %
    for j in range(0, species_list.size()):#species in species_list:
        amounts[j] = state.speciesAmount(species_list[j].name())[0] / bU * 100
    # Update dataframe with obtained values
    df.loc[len(df)] = np.concatenate([[pH], amounts])

    # If the equilibrium calculations didn't succeed, continue to the next condition
    if not res.optima.succeeded: continue

In the following, the obtained values of U(VI) speciation in % are presented in two diagrams. At the top, the species with a higher range of values are shown. Below, the U(VI) speciation of species with lower values is also shown.

In [6]:
species_high = "UO2+2 UO2OH+ UO2CO3@ UO2(CO3)3-4 UO2(CO3)2-2".split()
species_low = "UO2(OH)2@ (UO2)3CO3(OH)3+ (UO2)2(OH)+3 (UO2)2(OH)2+2 (UO2)3(OH)5+ (UO2)4(OH)7+".split()

from bokeh.plotting import gridplot, figure, show
from bokeh.io import output_notebook
output_notebook()

p1 = figure(
    title="",
    x_axis_label=r'PH [-]',
    y_axis_label='U(VI) SPECIATION [%]',
    sizing_mode="scale_width",
    plot_height=300)

colors1 = ['teal', 'darkred', 'indigo', 'coral', 'rosybrown', 'steelblue']
for species, color in zip(species_high, colors1):
    p1.line("pH", "amount_"+species, legend_label=species, line_width=3, line_cap="round", line_color=color, source=df)
p1.legend.location = 'center_right'

p2 = figure(
    title="",
    x_axis_label=r'PH [-]',
    y_axis_label='U(VI) SPECIATION [%]',
    sizing_mode="scale_width",
    plot_height=300)

colors2 = ['seagreen', 'palevioletred', 'darkred', 'darkkhaki', 'cadetblue', 'indianred']
for species, color in zip(species_low, colors2):
    p2.line("pH", "amount_"+species, legend_label=species, line_width=3, line_cap="round", line_color=color, source=df)
p2.legend.location = 'center_right'

grid = gridplot([[p1], [p2]])

show(grid)

Loading BokehJS ...